<a href="https://colab.research.google.com/github/talhaanwarch/Urdu-Sentiment-Corpus/blob/master/SimpleBaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,f1_score
import collections
import numpy as np
import csv
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [0]:
url='https://raw.githubusercontent.com/MuhammadYaseenKhan/Urdu-Sentiment-Corpus/master/urdu-sentiment-corpus-v1.tsv'
data=pd.read_csv(url,sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')

In [6]:
print('rows in data ',len(data))

rows in data  1000


In [5]:
data.head()

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P


In [0]:
data["Tweet"]=data['Tweet'].str.replace('/d+','') #remove numbers
#we will do other preprocessing latter

In [0]:
tweets=data['Tweet']
labels=le.fit_transform(data['Class'])

In [12]:
collections.Counter(data['Class'])


Counter({'N': 500, 'O': 20, 'P': 480})

#Classification

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()


#GridSearchCV

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.1, 1,3,5,8, 10,12,15]
    param_grid = {'C': C}
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=nfolds,scoring='f1_weighted')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_

train_cv = count_vectorizer.fit_transform(tweets)
logistic_param_selection(train_cv,labels,5)

{'C': 15}


0.608408981514869

we got macro f1 score of .41 which is quite low, now we will remove O class and classifiy the other two, inorder to avoid data imabalance

In [0]:
new_data=data[data['Class']!='O']

In [0]:
new_tweets=new_data['Tweet']
new_labels=le.fit_transform(new_data['Class'])

In [22]:
collections.Counter(new_data['Class'])

Counter({'N': 500, 'P': 480})

In [28]:
count_vectorizer = CountVectorizer()
train_cv = count_vectorizer.fit_transform(new_tweets)
logistic_param_selection(train_cv,new_labels,5)

{'C': 10}


0.6313793638578384

#TFIDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)

In [31]:
train_tfidf=word_vectorizer.fit_transform(new_tweets)
logistic_param_selection(train_tfidf,new_labels,5)

{'C': 3}


0.6200804962349146

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)


In [34]:
train_tfidfc=char_vectorizer.fit_transform(new_tweets)
logistic_param_selection(train_tfidfc,new_labels,5)

{'C': 15}


0.6510796603560024